In [1]:
from profit_maximization import *
import pickle

# random.seed(1000)  # for replicability of experiments.

instance_base = generate_base_instance(20)
coin_flip_params = get_coin_flip_params()
instance = flip_coins_for(0,instance_base,coin_flip_params)
get_stats(0,instance)
experiment_params = {'DISCOUNT_SETTING':'detour_based','GAMMA':0}
solution,total_profit = solve_instance(0,instance,experiment_params)
print "total profit:",total_profit
for k in solution:
    print(k,solution[k])

Coeff 100: Coin flip 0: Stats: Requests in provider market share w/o Gamma: 13
Coeff 100: Coin flip 0: Stats: Requests NOT in provider market share w/o Gamma: 7
Coeff 100: Coin flip 0: Experiment Gamma = 0. Total profit: 18610.3643011. Size of matching graph: 6. #Total ridesharers: 6
total profit: 18610.3643011
('unmatched_requests', [18, 1])
('matching_graph', <networkx.classes.graph.Graph object at 0x0000000003B415F8>)
('matched_request_pairs_with_permutations', {(5, 15): ('sj', 'si', 'di', 'dj'), (10, 19): ('si', 'sj', 'dj', 'di')})
('non_ridesharing_requests', [0, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 16, 17])


In [2]:
import matplotlib.pyplot as plt

data = {x:instance['all_requests'][x] for x in instance['all_requests'] if instance['all_requests'][x]['RIDE_SHARING'][experiment_params['GAMMA']]==True}

N = len(data)
labels = ['{0}'.format(i) for i in data]
plt.subplots_adjust(bottom = 0.1)
pd = {'orig':'green','dest':'red'}
for loc in pd:
    x = [data[z][loc][0] for z in data]
    y = [data[z][loc][1] for z in data]

#     plt.scatter(x, y, s=80, c=pd[loc], marker="o")

    plt.scatter(x, y, marker = 'o', 
        c = pd[loc], s = 1500,
        cmap = plt.get_cmap('Spectral'))
    for label, x, y in zip(labels, x, y):
        plt.annotate(
            label+','+loc, 
            xy = (x, y), xytext = (-20, 20),
            textcoords = 'offset points', ha = 'right', va = 'bottom',
            bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5),
            arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))

plt.show()

C:\Users\q4fj4lj9\AppData\Local\Continuum\Miniconda\lib\site-packages\matplotlib\collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [3]:
r1 = 10
r2 = 19
print get_profit_unmatched([r1],instance)
print get_profit_unmatched([r2],instance)
print get_incremental_profit([r1,r2],instance,experiment_params)

707.742891169
391.152144312
[2092.4554675311979, ('si', 'sj', 'dj', 'di')]
